<h1 style='text-align:center;color:Blue'> Drowsiness Detection </h1>

## Import Libraries

In [17]:
import imutils
import cv2
import dlib
import time
import playsound
import os
import numpy as np
from imutils import face_utils
from scipy.spatial import distance as dist
from threading import Thread

## Defining Face Detector and Localization points predictor

In [18]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

## EAR Function [Details](http://vision.fe.uni-lj.si/cvww2016/proceedings/papers/05.pdf)

In [32]:
def play_sound():
    file = 'music2.mp3'
    os.system("mpg123 " + file)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1],eye[5])
    B = dist.euclidean(eye[2],eye[4])
    C = dist.euclidean(eye[0],eye[3])
    ear = (A+B)/(2.0*C)
    return ear

## Defining Threshold and Max Frames

In [33]:
EYE_AR_THRESHOLD = 0.3
EYE_MAX_FRAMES = 48
Counter = 0
Total = 0

## Start image Capture

In [34]:
vid = cv2.VideoCapture(0)
time.sleep(1.0)#for camera to become ready
(lstart,lend) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart,rend) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

## Info
- Loop through every frame
- Find faces
- Locate landmarks
- Saperate Lefteye,Righteye landmarks
- Caluculate EAR
- Check if it's less than threshold and is been for threshold frames
- Trigger Alarm

In [35]:
AlarmOn=False
while True:
    _,frame = vid.read()
    if(_):
        frame = imutils.resize(frame,width=500)
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        rects = detector(gray,0)
        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lstart:lend]
            rightEye = shape[rstart:rend]

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR+rightEAR)/2.0

            leftHull = cv2.convexHull(leftEye)
            rightHull = cv2.convexHull(rightEye)

            cv2.drawContours(frame,[leftHull],-1,(0,0,255),1)
            cv2.drawContours(frame,[rightHull],-1,(0,0,255),1)

            if ear < EYE_AR_THRESHOLD:
                Counter+=1

                if Counter >= EYE_MAX_FRAMES:
                    if not AlarmOn:
                        AlarmOn = True
                        t = Thread(target=play_sound)
                        t.deamon = True
                        t.start()
                        #Put your audio here
                    cv2.putText(frame,'Drowsiness Alert',(10,30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            else:
                Counter = 0
                AlarmOn = False
            cv2.putText(frame,'{:.1f}'.format(ear),(300,30),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        
vid.release()
cv2.destroyAllWindows()